# Wave Polarization

This notebook provides an animated visualization of wave polarization. The code below implements the following function:

$E(z,t) = (E_{x0}~\hat{i} + E_{y0} e^{j\phi}~\hat{j}) e^{-\alpha z} e^{j (\omega t - \beta z)}$

(See equation 93 from textbook Ch 11 for a lossless phasor form of this equation.)

The simulation parameters are initially set as follows:
* `z`: $z$ position to animate.
* `phi`: $\phi$ in the expression above denoting a phase difference between $E_{x0}$ and $E_{y0}$. Set to $0$ initially.
* `E_x0` and `E_y0`: $E_{x0} = E_{y0} = 1$, you are free to change them.
* `alpha`: attenuation coefficient in the medium, set to $0$ initially.
* `lmda`: wavelength $\lambda$, set to $1~\text{m}$.
* `freq`: linear frequency $f$, set to $1~\text{Hz}$.

You are free to change _any_ of the simulation parameters.

Some things to explore:
* How do you achieve clockwise circular polarization? How about counterclockwise?
* What about elliptical polarization?

In [9]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.widgets import Slider
from IPython.display import HTML

# set plot params
x_min, x_max = -2, 2
y_min, y_max = -2, 2
x_res = 250     # this many discrete points per unit x

# animation timing settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 5    # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)
t_vals = np.linspace(0, anim_period, anim_frames)

In [18]:
# use np.pi to access pi
z = 0
phi = np.pi/2
E_x0, E_y0 = 15, -5
alpha = 0
lmda = 1
freq = 1

def vec_func(z, t):
    '''
    Return the vector end point at time t as a tuple.
    '''
    omega = 2*np.pi*freq
    beta = 2*np.pi/lmda
    
    E = np.array([E_x0, E_y0 * np.exp(np.complex(0, phi))]) \
            * np.exp(np.complex(-alpha*z, omega*t-beta*z))
    return E[0].real, E[1].real


In [19]:
def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    #[l.set_data([], []) for l in lines.values()]
    #return (l for l in lines.values())
    line.set_data([], [])
    quiver.set_UVC(0, 0)
    return quiver, line

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    #z = np.linspace(x_min, x_max, x_res*x_max)
    #[lines[comp].set_data(z,wave_func(z,t,refl_coef,comp)) for comp in ['a','i','r']]
    #return (l for l in lines.values())
    vec = vec_func(z, t)
    line_x.append(vec[0])
    line_y.append(vec[1])
    line.set_data(line_x, line_y)
    quiver.set_UVC(*vec)
    return quiver, line

# set up plotting canvas
line_x = []
line_y = []
fig, ax = plt.subplots(figsize=(7,7))
ax.set_title('Phasor')
ax.set_xlabel(r'$z$ (m)')
ax.set_ylabel(r'$V$ (V)')
ax.set_xlim((x_min,x_max))
ax.set_ylim((y_min,y_max))
ax.axvline(x=0, c='black', zorder=0)
ax.axhline(y=0, c='black', zorder=0)
quiver = ax.quiver(0, 0, 1, 1, angles='xy', scale_units='xy', scale=1, lw=2, color='tab:blue', zorder=2)
line, = ax.plot([], [], '.', color='tab:orange', markersize=4, zorder=1)
plt.close() # prevent empty plot from showing up
             
# generate animation
anim = animation.FuncAnimation(fig, animate_line, init_func=init,
                              frames=t_vals, interval=anim_interval, blit=True)
HTML(anim.to_html5_video())